In [13]:
import os
import requests
from PIL import Image
import io
import base64
import time

In [14]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path)
            img = img.resize((700, 700))  # Resize the image
            if img is not None:
                images.append((img, filename))
    return images

def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def get_cell_description(cell_type):
    cell_descriptions = {
        "cell_type_A549": "human lung adenocarcinoma alveolar basal epithelial cells",
        "cell_type_CACO-2": "colorectal adenocarcinoma epithelial cells",
        "cell_type_HPMEC": "human pulmonary microvascular endothelial cells",
        "cell_type_HSAEC": "human small airway epithelial cells",
        "cell_type_HUVEC": "human umbilical vein endothelial cells",
        "cell_type_NHBE": "normal human bronchial epithelial cells"
    }
    return cell_descriptions.get(cell_type, "Unknown cell type")

In [15]:
def load_and_resize_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path)
            img_resized = img.resize((512, 512))  # Resize to the default API width and height
            if img_resized is not None:
                images.append((img_resized, filename))
    return images

def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def generate_synthetic_images(base_folder, output_base, webui_url):
    categories = ['bad', 'good']
    cell_types_good = ['cell_type_A549', 'cell_type_CACO', 'cell_type_HPMEC', 'cell_type_HSAEC', 'cell_type_HUVEC', 'cell_type_NHBE']
    #cell_types_good = ['cell_type_HSAEC', 'cell_type_HUVEC', 'cell_type_NHBE']
    cell_types_bad = ['cell_type_A549', 'cell_type_CACO', 'cell_type_HPMEC', 'cell_type_HSAEC', 'cell_type_HUVEC', 'cell_type_NHBE']
    time_periods = ['0-1_days', '2-3_days', '4+_days', '4_days']

    for category in categories:
        if category == 'bad':
            cell_types = cell_types_bad
        else:  # category == 'good'
            cell_types = cell_types_good

        for cell_type in cell_types:
            for period in time_periods:
                real_path = os.path.join(base_folder, category, cell_type, period)
                synthetic_path = os.path.join(output_base, category, cell_type, period)
                ensure_dir(synthetic_path)
                if os.path.exists(real_path):
                    images = load_and_resize_images_from_folder(real_path)

                    for img_resized, filename in images:
                        img_byte_array = io.BytesIO()
                        img_resized.save(img_byte_array, format='PNG')
                        base64_encoded_image = base64.b64encode(img_byte_array.getvalue()).decode('utf-8')
                        img_byte_array.seek(0)  # Reset byte array position to the beginning
                        
                        for idx in range(3):  # Generate 3 synthetic images per original image
                            data = {
                                "prompt": f"{category} cells age: {period}",
                                "steps": 20,
                                "cfg_scale": 29,
                                "denoising_strength": 0.1, # change this 
                                "width": 512,
                                "height": 512,
                                "init_images": [base64_encoded_image]
                            }
                            headers = {'Content-Type': 'application/json'}
                            response = requests.post(f"{webui_url}/sdapi/v1/img2img", json=data)
                            if response.status_code == 200:
                                for img_data in response.json()['images']:
                                    img_bytes = base64.b64decode(img_data)
                                    synthetic_image = Image.open(io.BytesIO(img_bytes))
                                    synthetic_filename = f"synthetic_{idx+1}_{filename}"
                                    synthetic_image.save(os.path.join(synthetic_path, synthetic_filename))

                                for img_data in response.json()['info']:
                                    print(img_data)
                            else:
                                print("Error:", response.status_code)
            #time.sleep(600)


In [16]:


# Define the base directory where your original images are stored
base_folder = 'OOC_image_dataset/train'

# Define the output base directory where synthetic images should be saved
output_base = 'synthetic_data_0.2/train'

# URL of your running Stable Diffusion WebUI
webui_url = 'http://127.0.0.1:7860'

# Call the function with the specified parameters
generate_synthetic_images(base_folder, output_base, webui_url)

{
"
p
r
o
m
p
t
"
:
 
"
b
a
d
 
c
e
l
l
s
 
a
g
e
:
 
0
-
1
_
d
a
y
s
"
,
 
"
a
l
l
_
p
r
o
m
p
t
s
"
:
 
[
"
b
a
d
 
c
e
l
l
s
 
a
g
e
:
 
0
-
1
_
d
a
y
s
"
]
,
 
"
n
e
g
a
t
i
v
e
_
p
r
o
m
p
t
"
:
 
"
"
,
 
"
a
l
l
_
n
e
g
a
t
i
v
e
_
p
r
o
m
p
t
s
"
:
 
[
"
"
]
,
 
"
s
e
e
d
"
:
 
8
1
5
2
1
1
3
5
2
,
 
"
a
l
l
_
s
e
e
d
s
"
:
 
[
8
1
5
2
1
1
3
5
2
]
,
 
"
s
u
b
s
e
e
d
"
:
 
2
9
2
6
7
9
3
8
6
6
,
 
"
a
l
l
_
s
u
b
s
e
e
d
s
"
:
 
[
2
9
2
6
7
9
3
8
6
6
]
,
 
"
s
u
b
s
e
e
d
_
s
t
r
e
n
g
t
h
"
:
 
0
,
 
"
w
i
d
t
h
"
:
 
5
1
2
,
 
"
h
e
i
g
h
t
"
:
 
5
1
2
,
 
"
s
a
m
p
l
e
r
_
n
a
m
e
"
:
 
"
E
u
l
e
r
"
,
 
"
c
f
g
_
s
c
a
l
e
"
:
 
2
9
.
0
,
 
"
s
t
e
p
s
"
:
 
2
0
,
 
"
b
a
t
c
h
_
s
i
z
e
"
:
 
1
,
 
"
r
e
s
t
o
r
e
_
f
a
c
e
s
"
:
 
f
a
l
s
e
,
 
"
f
a
c
e
_
r
e
s
t
o
r
a
t
i
o
n
_
m
o
d
e
l
"
:
 
n
u
l
l
,
 
"
s
d
_
m
o
d
e
l
_
n
a
m
e
"
:
 
"
v
1
-
5
-
p
r
u
n
e
d
-
e
m
a
o
n
l
y
"
,
 
"
s
d
_
m
o
d
e
l
_
h
a
s
h
"
:
 
"
6
c
e
0
1
6
1
6
8
9
"
,
 
"
s
d
_
v
a
e
_
n
a
m
e
"
:


KeyboardInterrupt: 